In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

### 파일 병합 및 VIF 값 확인 후 다중공선성 유발 컬럼 제거

In [2]:
# VIF > 10이면 한 개씩 제거하며 반복, 마지막에 제거된 전체 리스트 출력
def remove_high_vif(df, id_col='ID', thresh=10):
    # ID 컬럼 제외한 숫자형 리스트
    cols = [c for c in df.select_dtypes(include='number').columns if c != id_col]
    removed = []
    while True:
        # 결측치 0으로 채우고 상수항 추가
        X = df[cols].fillna(0)
        X_const = sm.add_constant(X)
        # 각 컬럼별 VIF 계산
        vif_vals = [
            variance_inflation_factor(X_const.values, i+1)
            for i in range(len(cols))
        ]
        vif_df = (
            pd.DataFrame({'feature': cols, 'vif': vif_vals})
              .sort_values('vif', ascending=False)
              .reset_index(drop=True)
        )
        # 가장 높은 VIF가 임계치 이하이면 종료
        if vif_df.loc[0, 'vif'] <= thresh:
            break
        # 넘는 경우, 그 컬럼 제거
        to_remove = vif_df.loc[0, 'feature']
        removed.append(to_remove)
        cols.remove(to_remove)
    # 한꺼번에 제거된 컬럼 리스트 출력
    print("제거된 VIF>10 컬럼:", removed)
    # 실제 DataFrame에서 삭제 후 반환
    return df.drop(columns=removed), removed

In [3]:
# 데이터 가져오기
marketing_df   = pd.read_csv('마케팅정보_최종.csv')
performance_df, perf_removed = remove_high_vif(pd.read_csv('성과정보12월_feat_select.csv'))
credit_df,      credit_removed = remove_high_vif(pd.read_csv('신용정보12월_feat_select.csv'))
channel_df,     channel_removed = remove_high_vif(pd.read_csv('채널정보_최종.csv'))
member_df,      member_removed = remove_high_vif(pd.read_csv('회원정보12월_feat_select.csv'))
balance_df,     balance_removed = remove_high_vif(pd.read_csv('charge_balance_201812.csv'))
approval_df,    approval_removed = remove_high_vif(pd.read_csv('승인매출정보_최종.csv'))

# 예비 제거 결과 출력
print("performance removed:", perf_removed)
print("credit removed     :", credit_removed)
print("channel removed    :", channel_removed)
print("member removed     :", member_removed)
print("balance removed    :", balance_removed)
print("approval removed   :", approval_removed)

제거된 VIF>10 컬럼: ['잔액_신판ca최대한도소진율_r3m', '증감율_이용금액_신판_전월', '잔액_신판ca평균한도소진율_r6m', '변동률_RVCA평잔', '혜택수혜율_R3M']
제거된 VIF>10 컬럼: ['카드이용한도금액_B1M', '카드이용한도금액', '일시불ONLY전환가능여부', '상향가능한도금액', 'rv최초시작후경과일', 'RV현금서비스이자율_할인전']
제거된 VIF>10 컬럼: ['방문후경과월_PC_R6M', '인입일수_ARS_R6M', '홈페이지_금융건수_R6M', '방문월수_PC_R6M', '인입일수_ARS_B0M', '방문월수_앱_R6M']
제거된 VIF>10 컬럼: ['입회일자_신용', '이용금액_R3M_신용체크', '이용가능카드수_신용']
제거된 VIF>10 컬럼: ['마일_적립포인트_R3M', '할인금액_R3M', '청구금액_R3M', '포인트_이용포인트_R3M', '할인금액_청구서_R3M', '혜택수혜금액', '상환개월수_결제일_R6M', '포인트_마일리지_환산_B0M']
제거된 VIF>10 컬럼: ['이용금액_체크_R3M', '이용금액_오프라인_R3M', '이용금액_오프라인_R6M', '정상청구원금_B0M', '이용금액_체크_R6M', '이용개월수_신용_R12M']
performance removed: ['잔액_신판ca최대한도소진율_r3m', '증감율_이용금액_신판_전월', '잔액_신판ca평균한도소진율_r6m', '변동률_RVCA평잔', '혜택수혜율_R3M']
credit removed     : ['카드이용한도금액_B1M', '카드이용한도금액', '일시불ONLY전환가능여부', '상향가능한도금액', 'rv최초시작후경과일', 'RV현금서비스이자율_할인전']
channel removed    : ['방문후경과월_PC_R6M', '인입일수_ARS_R6M', '홈페이지_금융건수_R6M', '방문월수_PC_R6M', '인입일수_ARS_B0M', '방문월수_앱_R6M']
member removed     : ['입회일자_신용', '이용

In [4]:
# 보조 데이터프레임에서 Segment 제거
dfs = [performance_df, credit_df, channel_df, member_df, balance_df, approval_df]
for df in dfs:
    if 'Segment' in df.columns:
        df.drop(columns=['Segment'], inplace=True)

# 순차 병합
merged_df = marketing_df.copy()
for df in dfs:
    merged_df = (
        merged_df
        .merge(df, on='ID', how='left')
        .drop_duplicates(subset='ID')
    )

# 기준년월 제거
if '기준년월' in merged_df.columns:
    merged_df.drop(columns=['기준년월'], inplace=True)

In [5]:
# 병합 전 전체 컬럼 수 확인
print("병합 후 전체 컬럼 수:", merged_df.shape[1])

병합 후 전체 컬럼 수: 118


In [7]:
# 병합 후 VIF 확인 및 제거
merged_df, merge_removed = remove_high_vif(merged_df)
print("병합 후 제거된 칼럼:", merge_removed)

제거된 VIF>10 컬럼: ['포인트_적립포인트_R3M', '카드론동의여부', '최대이용금액_체크_R12M', '잔액_신판ca최대한도소진율_r6m', '_1순위카드이용금액', '캠페인접촉건수_R12M_num', '정상청구원금_B2M', '회원여부_이용가능_카드론', '컨택건수_이용유도_EM_R6M']
병합 후 제거된 칼럼: ['포인트_적립포인트_R3M', '카드론동의여부', '최대이용금액_체크_R12M', '잔액_신판ca최대한도소진율_r6m', '_1순위카드이용금액', '캠페인접촉건수_R12M_num', '정상청구원금_B2M', '회원여부_이용가능_카드론', '컨택건수_이용유도_EM_R6M']


In [10]:
# TRAIN/Test 데이터 분리
train_df = merged_df[merged_df['ID'].str.startswith('TRAIN')]
test_df  = merged_df[merged_df['ID'].str.startswith('TEST')]

# 분리 결과 확인
display(train_df.head())
display(test_df.head())

,ID,Segment,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_이용유도_LMS_R6M,컨택건수_이용유도_LMS_B0M,캠페인접촉일수_R12M_num,컨택건수_이용유도_청구서_R6M,...,마일_잔여포인트_B0M,포인트_포인트_월적립_R3M,이용금액대_ord,연속유실적개월수_기본_24M_카드,최대이용금액_CA_R12M,이용금액_오프라인_B0M,이용금액_체크_R12M,이용개월수_일시불_R6M,이용건수_신용_R12M,이용금액_일시불_R12M
0,TRAIN_000000,D,0,0,13,0,12,2,1,0,...,0,0,6,17,12264,3931,7824,6,147,24782
1,TRAIN_000001,E,0,0,0,3,4,2,15,0,...,0,0,5,17,3516,4033,-414,6,177,53959
2,TRAIN_000002,C,0,0,1,0,0,0,1,0,...,0,0,6,8,69186,10536,-414,6,149,60220
3,TRAIN_000003,D,0,0,2,0,12,2,1,1,...,0,0,6,24,9802,3940,-414,6,107,16649
4,TRAIN_000004,E,1,3,0,0,14,2,1,3,...,0,0,2,0,0,0,12988,1,-1,-861


,ID,Segment,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_TM_R6M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_인터넷_R6M,컨택건수_이용유도_LMS_R6M,컨택건수_이용유도_LMS_B0M,캠페인접촉일수_R12M_num,컨택건수_이용유도_청구서_R6M,...,마일_잔여포인트_B0M,포인트_포인트_월적립_R3M,이용금액대_ord,연속유실적개월수_기본_24M_카드,최대이용금액_CA_R12M,이용금액_오프라인_B0M,이용금액_체크_R12M,이용개월수_일시불_R6M,이용건수_신용_R12M,이용금액_일시불_R12M
400000,TEST_00000,NaN,0,0,1,2,12,2,1,0,...,0,0,3,8,0,9741,1890,6,110,62221
400001,TEST_00001,NaN,0,0,2,0,0,0,5,0,...,0,0,3,8,8768,3490,47520,6,54,7081
400002,TEST_00002,NaN,0,0,2,2,12,2,15,0,...,0,6032,3,24,0,10759,-414,6,525,71644
400003,TEST_00003,NaN,0,0,1,3,0,0,15,0,...,0,0,4,9,0,965,144,6,317,8418
400004,TEST_00004,NaN,0,0,0,0,12,2,1,0,...,0,0,4,17,0,985,6664,6,182,15258


In [11]:
# train_df 저장
train_df.to_csv('final_train.csv', index=False, encoding='utf-8-sig')

# test_df 저장
test_df.to_csv('final_test.csv', index=False, encoding='utf-8-sig')